# Quantum Support Vector Machines: Two Moons Evaluation

In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
  %pip -q install "qiskit>=1.2" "qiskit-machine-learning>=0.7" "qiskit-algorithms>=0.3"

In [ ]:
if IN_COLAB:
  # ensure directory is importable
  sys.path.append("/content/quantum-support-vector-machines")

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from svm import ClassicSVM
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from qiskit.circuit.library import ZZFeatureMap

from qsvm import (
    QuantumSVM,
    FidelityKernel,
    ProjectedKernel
)


from evaluation_helpers import (
    decision_boundary_plot,
    compose_boundary_grid_from_files,
    plot_confusion_matrices_row,
)

RNG = 42
np.random.seed(RNG)

# paths for saved plots
EVAL_DIR = Path("evaluations") / "two_moons"
EVAL_DIR.mkdir(parents=True, exist_ok=True)

PLOT_LINEAR = EVAL_DIR / "linear_boundary.png"
PLOT_RBF = EVAL_DIR / "rbf_boundary.png"
PLOT_QSVM = EVAL_DIR / "qsvm_boundary.png"
PLOT_PQ = EVAL_DIR / "projected_qsvm_boundary.png"

In [ ]:
data, labels = make_moons(n_samples=600, noise=0.20, random_state=RNG)

# train/test split
x_train, x_test, y_train, y_test = train_test_split(
    data, labels, test_size=0.30, stratify=labels, random_state=RNG
)

In [ ]:
# train
lin_svm = ClassicSVM(SVC(kernel="linear", C=1.0, random_state=RNG)).fit(x_train, y_train)

# plot decision boundary
fig_lin = decision_boundary_plot(
    predict_fn=lambda G: lin_svm.predict(G),
    X=data, y=labels, save_path=PLOT_LINEAR,
    title="Linear SVM",
    n=150
)

# eval
preds_lin_svm = lin_svm.predict(x_test)
acc_lin = accuracy_score(y_test, preds_lin_svm)
print(f"Linear SVM accuracy: {acc_lin:.3f}")

# confusion matrix
ConfusionMatrixDisplay(confusion_matrix(y_test, preds_lin_svm)).plot(colorbar=False)
plt.title("Linear SVM — Confusion Matrix")
plt.show()


In [ ]:
# train
rbf_svm = ClassicSVM(SVC(kernel="rbf", C=1.0, gamma="scale", random_state=RNG)).fit(x_train, y_train)

# plot decision boundary
fig_rbf = decision_boundary_plot(
    predict_fn=lambda G: rbf_svm.predict(G),
    X=data, y=labels, save_path=PLOT_RBF,
    title="RBF SVM",
    n=150
)

# eval
preds_rbf_svm = rbf_svm.predict(x_test)
acc_rbf = accuracy_score(y_test, preds_rbf_svm)
print(f"RBF SVM accuracy: {acc_rbf:.3f}")

# confusion matrix
ConfusionMatrixDisplay(confusion_matrix(y_test, preds_rbf_svm)).plot(colorbar=False)
plt.title("RBF SVM — Confusion Matrix")
plt.show()


In [ ]:
# build a shallow feature map (2 features → 2 qubits)
feature_map = ZZFeatureMap(feature_dimension=2, reps=1, entanglement="linear")

# QSVM uses MinMax scaling internally; set to [0, 2π] for this feature map
qsvm_fqk = QuantumSVM(kernel=FidelityKernel(feature_map), C=1.0, scale_range=(0.0, 2*np.pi), seed=RNG)
qsvm_fqk.fit(x_train, y_train)

# plot decision boundary
sv_count_q = len(qsvm_fqk._clf.support_)
fig_qsvm = decision_boundary_plot(
    predict_fn=lambda G: qsvm_fqk.predict(G, batch_size=1024),
    X=data, y=labels, save_path=PLOT_QSVM,
    title="QuantumSVM Fidelity Kernel",
    budget=500_000, sv_count=sv_count_q, min_n=35
)

# eval
preds_qsvm_fqk = qsvm_fqk.predict(x_test, batch_size=1024)
acc_pqk = accuracy_score(y_test, preds_qsvm_fqk)
print(f"Fidelity QSVM accuracy: {acc_pqk:.3f}")

# confusion matrix
ConfusionMatrixDisplay(confusion_matrix(y_test, preds_qsvm_fqk)).plot(colorbar=False)
plt.title("Fidelity QSVM — Confusion Matrix")
plt.show()


In [ ]:
proj_kernel = ProjectedKernel(feature_map, gamma=1.0)
qsvm_pqk = QuantumSVM(kernel=proj_kernel, C=1.0, scale_range=(0.0, 2*np.pi), seed=RNG)

# train
qsvm_pqk.fit(x_train, y_train)

# plot decision boundary
sv_count_p = len(qsvm_pqk._clf.support_)
fig_proj = decision_boundary_plot(
    predict_fn=lambda G: qsvm_pqk.predict(G, batch_size=1024),
    X=data, y=labels, save_path=PLOT_PQ,
    title="QuantumSVM Projected Kernel",
    budget=500_000, sv_count=sv_count_p, min_n=35
)

# eval
preds_qsvm_pqk = qsvm_pqk.predict(x_test, batch_size=1024)
acc_pqk = accuracy_score(y_test, preds_qsvm_pqk)
print(f"Projected QSVM accuracy: {acc_pqk:.3f}")

# confusion matrix
ConfusionMatrixDisplay(confusion_matrix(y_test, preds_qsvm_pqk)).plot(colorbar=False)
plt.title("Projected QSVM — Confusion Matrix")
plt.show()


In [ ]:
# accuracy summary
acc_table = {
    "Linear SVM": acc_lin,
    "RBF SVM": acc_rbf,
    "Fidelity QSVM": acc_fqk,
    "Projected QSVM": acc_pqk,
}
print("Final accuracies:")
for k, v in acc_table.items():
    print(f"  {k:18s} : {v:.3f}")

# confusion matrices row (normalized=None for raw counts; use 'true' for per-class rates)
_ = plot_confusion_matrices_row(
    y_true=y_test,
    y_preds=[preds_lin_svm, preds_rbf_svm, preds_qsvm_fqk, preds_qsvm_pqk],
    titles=["Linear SVM", "RBF SVM", "Fidelity QSVM", "Projected QSVM"],
    save_path=str(EVAL_DIR / "confusion_matrices_1x4.png"),
    normalize=None,
    include_colorbar=False,
    figsize=(18, 3.8),
)
plt.show()


_ = compose_boundary_grid_from_files(
    image_paths=[PLOT_LINEAR, PLOT_RBF, PLOT_QSVM, PLOT_PQ],
    save_path=str(EVAL_DIR / "boundaries_2x2.png"),
    titles=["Linear SVM", "RBF SVM", "Fidelity QSVM", "Projected QSVM"],
    cols=2, dpi=150,
)
plt.show()
